# Federated Learning Implementation with tensorflow



In [1]:
# """
# Adult Dataset Salary Prediction 
# This is part of a study to investigate Differetinal privacy in Machine learning, Naturally we wish to compare it with federated learning.



# Refrences:

# [1] Federated Learning with Non-IID Data, Yue Zhao et al, arXiv: 1806.00582v1, 2 Jun 2018
# [2] Communication-Efficient Learning of Deep Networks from Decentralized Data, H. Brendan McMahan et al, arXiv:1602.05629v3 [cs.LG] 28 Feb 2017

# """

In [2]:
# !pip install --upgrade pip

In [3]:
# !pip uninstall numpy
# !pip install numpy==1.21.4
# !pip install scipy

In [4]:
# np.__version__
# !pip uninstall numpy

In [5]:
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers
from sklearn.metrics import precision_score,recall_score, accuracy_score,confusion_matrix,f1_score
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import History

import pandas as pd



C:\Users\riash\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\riash\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\riash\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\riash\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [6]:
### Train/Test Data seperation

file_out = pd.read_csv('Data/adult_processed.csv')
cols = []
for i in list(file_out.columns):
    if  i != 'income':
        cols.append(i)

feature_set1 = pd.read_csv('Data/train.csv')
feature_set2 = pd.read_csv('Data/test.csv')

print(feature_set1['race_White'])
x = feature_set1[cols].copy().values
y = feature_set1[['income']].copy().values
        
sc = StandardScaler()
X_train = sc.fit_transform(x)
y_train = y

x2 = feature_set2[cols].copy().values
y2 = feature_set2[['income']].copy().values
        
X_test = sc.transform(x2)
y_test = y2


X_test.shape, y_test.shape 

0        1
1        1
2        1
3        1
4        1
        ..
39068    0
39069    1
39070    1
39071    1
39072    1
Name: race_White, Length: 39073, dtype: int64


((9768, 94), (9768, 1))

In [7]:
# Train/Test data based on  gender: 4 sets

# train

female_features =  feature_set1[feature_set1['gender_Female']==1]
x_female = female_features[cols].copy().values
y_female = female_features[['income']].copy().values

X_train_female = sc.fit_transform(x_female)
y_train_female = y_female

male_features = feature_set1[feature_set1['gender_Male']==1] 
x_male = male_features[cols].copy().values
y_male = male_features[['income']].copy().values

X_train_male = sc.fit_transform(x_male)
y_train_male = y_male

# test

female_features2 =  feature_set2[feature_set2['gender_Female']==1]
x_female2 = female_features2[cols].copy().values
y_female2 = female_features2[['income']].copy().values

X_test_female = sc.fit_transform(x_female2)
y_test_female = y_female2 

male_features2 = feature_set2[feature_set2['gender_Male']==1] 
x_male2 = male_features2[cols].copy().values
y_male2 = male_features2[['income']].copy().values

X_test_male = sc.fit_transform(x_male2)
y_test_male = y_male2


# checks
X_train.shape, X_train_female.shape, X_train_male.shape
# X_test.shape, X_test_female.shape, X_test_male.shape
# y_train.shape, y_train_female.shape, y_train_male.shape


((39073, 94), (12919, 94), (26154, 94))

In [8]:
# female_features_above30 = female_features[female_features['age']>30]
# x_female_age = female_features_above30[cols].copy().values
# y_female_age = female_features_above30[['income']].copy().values

# X_train_female_age = sc.fit_transform(x_female_age)
# y_train_female_age = y_female_age

# male_features_above30 = male_features[male_features['age']>30]
# x_male_age = male_features_above30[cols].copy().values
# y_male_age = male_features_above30[['income']].copy().values

# X_train_male_age = sc.fit_transform(x_male_age)
# y_train_male_age = y_male_age

# # test

# female_features_above30_2 =  female_features2[female_features2['age']>30]
# x_female_age2 = female_features_above30_2[cols].copy().values
# y_female_age2 = female_features_above30_2[['income']].copy().values

# X_test_female_age = sc.fit_transform(x_female_age2)
# y_test_female_age = y_female_age2 

# male_features_above30_2 = male_features2[male_features2['age']>30]
# x_male_age2 = male_features_above30_2[cols].copy().values
# y_male_age2 = male_features_above30_2[['income']].copy().values

# X_test_male_age = sc.fit_transform(x_male_age2)
# y_test_male_age = y_male_age2


# # checks
# X_train.shape, X_train_female_age.shape, X_train_male_age.shape
# X_test.shape, X_test_female.shape, X_test_male.shape
# y_train.shape, y_train_female.shape, y_train_male.shape

In [9]:

# def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    
#     ''' return: a dictionary with keys clients' names and value as 
#                 data shards - tuple of images and label lists.
#         args: 
#             image_list: a list of numpy arrays of training images
#             label_list:a list of binarized labels for each image
#             num_client: number of fedrated members (clients)
#             initials: the clients'name prefix, e.g, clients_1 
            
#     '''

#     #create a list of client names
# #     client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

# #     iid
# #     #randomize the data
# #     data = list(zip(image_list, label_list))
# #     random.shuffle(data)
    
   
    
# #     non-iid //////////////////////////////////////
#     max_y = np.argmax(label_list, axis=-1)
#     sorted_zip = sorted(zip(max_y, label_list, image_list), key=lambda x: x[0])
#     data = [(x,y) for _,y,x in sorted_zip]

# # LOGIC:
# #    image_list length-->total data items
# #    taking 50 images in each shard--> num_shards = image_list/41 gives 953 shards
# #    num_shard = num_shards + 1 if(image_list%50 >0) 

#     num_shards, num_imgs = 953, 41
#     idx_shard = [i for i in range(num_shards)]
#     dict_users = {i: np.array([]) for i in range(num_clients)}
# #     dict_users = [i for i in range(num_clients)]
# #     idxs = np.arange(num_shards*num_imgs)
# #     labels = dataset.train_labels.numpy()

#     max_y = np.argmax(label_list, axis=-1)
# #     image_list = np.argmax(image_list, axis=-1)
# #     print(idxs.shape, label_list.shape)
    
# #     idxs_labels = np.vstack((idxs, max_y))
# #     idxs_labels = idxs_labels[:, idxs_labels[1, :].argsort()]
# #     idxs = idxs_labels[0, :]


# #   ////////////////////////////////////////////////////  
#     #shard data and place at each client
# #     size = len(data)//num_clients
# #     shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

# #     #number of clients must equal number of shards
# #     assert(len(shards) == len(client_names))

# #     print(len(image_list))
# #     for i in range(len(client_names)):
# #         print(client_names[i], len(shards[i]))
              
# #     return {client_names[i] : shards[i] for i in range(len(client_names))} 


# clients = create_clients(X_train, y_train, num_clients=15, initial='client')
# # print(clients)

# for i in range(len(clients.keys())):
#         print(i, len(clients[i]))

# print(type(clients))

In [10]:
# gender split

dict_users = {i: np.array([]) for i in range(8)}
data_out = []

def create_hetero_clients( image_list, label_list, start_client = 0, num_clients=10, initial='clients'):
    
    selected_inds = []

    max_y = np.argmax(label_list, axis=-1)
    sorted_zip = sorted(zip(max_y, label_list, image_list), key=lambda x: x[0])
    data = [(x,y) for _,y,x in sorted_zip]
    
    data_out = data
    
    num_shards, num_imgs = int(len(image_list)/30), 30
    idx_shard = [i for i in range(num_shards)]

    min_shard = 1
    max_shard = 60  #953/15 = 63.53
    
    random_shard_size = np.random.randint(min_shard, max_shard+1,
                                          size=(num_clients-start_client))
    random_shard_size = np.around(random_shard_size /
                                  sum(random_shard_size) * num_shards)
    random_shard_size = random_shard_size.astype(int)


    if sum(random_shard_size) > num_shards:
        
        for i in range(start_client, num_clients):
            # First assign each client 1 shard to ensure every client has
            # atleast one shard of data
            rand_set = set(np.random.choice(idx_shard, 1, replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[i]) == 0):
                    dict_users[i] = data[ind1: ind2]
                else:
                    dict_users[i] = np.concatenate(
                    (dict_users[i],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])

        random_shard_size = random_shard_size-1

        # Next, randomly assign the remaining shards
        for i in range(start_client, num_clients):
            if len(idx_shard) == 0:
                continue
#             print(random_shard_size)
            shard_size = random_shard_size[i-start_client]
            if shard_size > len(idx_shard):
                shard_size = len(idx_shard)
            rand_set = set(np.random.choice(idx_shard, shard_size,
                                            replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[i]) == 0):
                    dict_users[i] = data[ind1: ind2]
                else:
                    dict_users[i] = np.concatenate(
                    (dict_users[i],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])

    else:

        for i in range(start_client, num_clients):
#             print(random_shard_size)
            shard_size = random_shard_size[i-start_client]
#             shard_size = random_shard_size[int(i/len(random_shard_size)) - 1]
            rand_set = set(np.random.choice(idx_shard, shard_size,
                                            replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[i]) == 0):
                    dict_users[i] = data[ind1: ind2]
                else:
                    dict_users[i] = np.concatenate(
                    (dict_users[i],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])

        if len(idx_shard) > 0:
            # Add the leftover shards to the client with minimum images:
            shard_size = len(idx_shard)
            # Add the remaining shard to the client with lowest data
            k = min(dict_users, key=lambda x: len(dict_users.get(x)))
            rand_set = set(np.random.choice(idx_shard, shard_size,
                                            replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[k]) == 0):
                    dict_users[k] = data[ind1: ind2]
                else:
                    dict_users[k] = np.concatenate(
                    (dict_users[k],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])
                
                
    return dict_users, selected_inds, data_out



In [11]:
# dict_users = {i: np.array([]) for i in range(4)}

def create_client_iid(image_list, label_list, client_num):    
    max_y = np.argmax(label_list, axis=-1)
    sorted_zip = sorted(zip(max_y, label_list, image_list), key=lambda x: x[0])
    data = [(x,y) for _,y,x in sorted_zip]
    
    dict_users[client_num] = data
    
    return dict_users

In [12]:
def get_hetero_clients():
    
    female_features =  feature_set1[feature_set1['gender_Female']==1]
    x_female = female_features[cols].copy().values
    y_female = female_features[['income']].copy().values

    X_train_female = sc.fit_transform(x_female)
    y_train_female = y_female

    male_features = feature_set1[feature_set1['gender_Male']==1] 
    x_male = male_features[cols].copy().values
    y_male = male_features[['income']].copy().values

    X_train_male = sc.fit_transform(x_male)
    y_train_male = y_male

    # test

    female_features2 =  feature_set2[feature_set2['gender_Female']==1]
    x_female2 = female_features2[cols].copy().values
    y_female2 = female_features2[['income']].copy().values

    X_test_female = sc.fit_transform(x_female2)
    y_test_female = y_female2 

    male_features2 = feature_set2[feature_set2['gender_Male']==1] 
    x_male2 = male_features2[cols].copy().values
    y_male2 = male_features2[['income']].copy().values

    X_test_male = sc.fit_transform(x_male2)
    y_test_male = y_male2

#     shuffling male and female datalists
#     print(X_train_male.shape)
    data = list(zip(X_train_male, y_train_male))
    random.shuffle(data)
    
    X_train_male, y_train_male = list(zip(*data))[0], list(zip(*data))[1]
    data.clear()
    
    data = list(zip(X_train_female, y_train_female))
    random.shuffle(data)
    
    X_train_female, y_train_female = list(zip(*data))[0], list(zip(*data))[1]
    
    
    
    # 1 all male
    clients, inds, data_out1 = create_hetero_clients(X_train_male[:5230], y_train_male[:5230], start_client = 0, num_clients=1, initial='client') 
    # 1 all female
    clients2, inds2, data_out2 = create_hetero_clients(X_train_female[:2584], y_train_female[:2584], start_client = 1, num_clients=2, initial='client')
    clients = {**clients, **clients2}

    used_x = []
    used_y = []
    used_data = []
  
    for i,j in list(inds):
        for x in range(i,j):
            used_data.insert(len(used_data)-1,(data_out1[x]))  
  
    for x in used_data:
        used_x.insert(len(used_x)-1,x[0])
        used_y.insert(len(used_y)-1,x[1])  
   
    used_data.clear()

    for i,j in list(inds2):
        for x in range(i,j):
            used_data.insert(len(used_data)-1,(data_out2[x]))  

    for x in used_data:
        used_x.insert(len(used_x)-1,x[0])
        used_y.insert(len(used_y)-1,x[1])  

    # 3 mix
    X_train_mix = np.setdiff1d(X_train, np.array(used_x))#53834, 64278
    y_train_mix = np.setdiff1d(y_train, np.array(used_y))

#     print(X_train_mix.shape, X_train.shape)
#     print(y_train_mix.shape, y_train.shape)

    clients3, inds3 , data_out= create_hetero_clients(X_train_mix, y_train_mix, start_client = 2, num_clients=5, initial='client')
    clients = {**clients, **clients3}
    
    return clients


In [13]:
def initializer():
    
    female_features =  feature_set1[feature_set1['gender_Female']==1]
    x_female = female_features[cols].copy().values
    y_female = female_features[['income']].copy().values

    X_train_female = sc.fit_transform(x_female)
    y_train_female = y_female

    male_features = feature_set1[feature_set1['gender_Male']==1] 
    x_male = male_features[cols].copy().values
    y_male = male_features[['income']].copy().values

    X_train_male = sc.fit_transform(x_male)
    y_train_male = y_male

    # test

    female_features2 =  feature_set2[feature_set2['gender_Female']==1]
    x_female2 = female_features2[cols].copy().values
    y_female2 = female_features2[['income']].copy().values

    X_test_female = sc.fit_transform(x_female2)
    y_test_female = y_female2 

    male_features2 = feature_set2[feature_set2['gender_Male']==1] 
    x_male2 = male_features2[cols].copy().values
    y_male2 = male_features2[['income']].copy().values

    X_test_male = sc.fit_transform(x_male2)
    y_test_male = y_male2
    
#   train-- with age
    female_features_above30 = female_features[female_features['age']>30]
    x_female_age = female_features_above30[cols].copy().values
    y_female_age = female_features_above30[['income']].copy().values

    X_train_female_age = sc.fit_transform(x_female_age)
    y_train_female_age = y_female_age

    male_features_above30 = male_features[male_features['age']>30]
    x_male_age = male_features_above30[cols].copy().values
    y_male_age = male_features_above30[['income']].copy().values

    X_train_male_age = sc.fit_transform(x_male_age)
    y_train_male_age = y_male_age

    # test

    female_features_above30_2 =  female_features2[female_features2['age']>30]
    x_female_age2 = female_features_above30_2[cols].copy().values
    y_female_age2 = female_features_above30_2[['income']].copy().values

    X_test_female_age = sc.fit_transform(x_female_age2)
    y_test_female_age = y_female_age2 

    male_features_above30_2 = male_features2[male_features2['age']>30]
    x_male_age2 = male_features_above30_2[cols].copy().values
    y_male_age2 = male_features_above30_2[['income']].copy().values

    X_test_male_age = sc.fit_transform(x_male_age2)
    y_test_male_age = y_male_age2
    
    return X_train_male_age, y_train_male_age,  X_train_female_age, y_train_female_age

In [14]:
def initializer_income():
    
    female_features =  feature_set1[feature_set1['gender_Female']==1]
    female_features_above30 = female_features[female_features['income']==1]
    x_female_age = female_features_above30[cols].copy().values
    y_female_age = female_features_above30[['income']].copy().values
    
    female_features_below = female_features[female_features['income']==0]
    x_female_less = female_features_below[cols].copy().values
    y_female_less = female_features_below[['income']].copy().values

    X_train_female_age = sc.fit_transform(x_female_age)
    y_train_female_age = y_female_age
    
    X_train_female_less = sc.fit_transform(x_female_less)
    y_train_female_less = y_female_less


    male_features = feature_set1[feature_set1['gender_Male']==1] 
    male_features_above30 = male_features[male_features['income']==1]
    x_male_age = male_features_above30[cols].copy().values
    y_male_age = male_features_above30[['income']].copy().values
    
    male_features_below = male_features[male_features['income']==0]
    x_male_less = male_features_below[cols].copy().values
    y_male_less = male_features_below[['income']].copy().values

    X_train_male_age = sc.fit_transform(x_male_age)
    y_train_male_age = y_male_age
    
    X_train_male_less = sc.fit_transform(x_male_less)
    y_train_male_less = y_male_less


    # test

    female_features2 =  feature_set2[feature_set2['gender_Female']==1]
    female_features_above30_2 =  female_features2[female_features2['income']==1]
    x_female_age2 = female_features_above30_2[cols].copy().values
    y_female_age2 = female_features_above30_2[['income']].copy().values

    X_test_female_age = sc.fit_transform(x_female_age2)
    y_test_female_age = y_female_age2 


    male_features2 = feature_set2[feature_set2['gender_Male']==1]
    male_features_above30_2 = male_features2[male_features2['income']==1]
    x_male_age2 = male_features_above30_2[cols].copy().values
    y_male_age2 = male_features_above30_2[['income']].copy().values

    X_test_male_age = sc.fit_transform(x_male_age2)
    y_test_male_age = y_male_age2

    
    return [X_train_male_age, y_train_male_age,  X_train_female_age, y_train_female_age, X_train_male_less, y_train_male_less,  X_train_female_less, y_train_female_less]

In [15]:
def get_hetero_clients_gender(train_sets):
    
    data = list(zip(train_sets[0], train_sets[1]))
    random.shuffle(data)
    
#     print(type(train_sets), type(train_sets[0]),type(list(list(zip(*data))[0]) ), type(data))
    
    train_sets[0] = list(list(zip(*data))[0])
    train_sets[1] = list(list(zip(*data))[1])
    data.clear()
    
    data = list(zip(train_sets[2], train_sets[3]))
    random.shuffle(data)
    
    train_sets[2], train_sets[3] = list(zip(*data))[0], list(zip(*data))[1]
    data.clear()
    
    data = list(zip(train_sets[4], train_sets[5]))
    random.shuffle(data)
    
    train_sets[4], train_sets[5] = list(zip(*data))[0], list(zip(*data))[1]
    data.clear()
    
    data = list(zip(train_sets[6], train_sets[7]))
    random.shuffle(data)
    
    train_sets[6], train_sets[7] = list(zip(*data))[0], list(zip(*data))[1]
    
#     NON_IID
    clients, inds, data_out1 = create_hetero_clients(train_sets[0], train_sets[1], start_client = 0, num_clients=1, initial='client') 
    
    clients2, inds2, data_out2 = create_hetero_clients(train_sets[2], train_sets[3], start_client = 1, num_clients=2, initial='client')
    clients = {**clients, **clients2}
    
    clients3, inds3, data_out3 = create_hetero_clients(train_sets[4], train_sets[5], start_client = 2, num_clients=3, initial='client')
    clients = {**clients, **clients3}
    
    clients4, inds4, data_out4 = create_hetero_clients(train_sets[6], train_sets[7], start_client = 3, num_clients=4, initial='client')
    clients = {**clients, **clients4}

# #     IID
#     clients = create_client_iid(train_sets[0], train_sets[1], 0)
    
#     clients2 = create_client_iid(train_sets[2], train_sets[3], 1)
#     clients = {**clients, **clients2}
    
#     clients3 = create_client_iid(train_sets[4], train_sets[5], 2)
#     clients = {**clients, **clients3}
    
#     clients4 = create_client_iid(train_sets[6], train_sets[7], 3)
#     clients = {**clients, **clients4}
    
    
#     ________________________________________________________
#     used_x = []
#     used_y = []
#     used_data = []
  
#     for i,j in list(inds):
#         for x in range(i,j):
#             used_data.insert(len(used_data)-1,(data_out1[x]))  
  
#     for x in used_data:
#         used_x.insert(len(used_x)-1,x[0])
#         used_y.insert(len(used_y)-1,x[1])  
   
#     used_data.clear()

#     for i,j in list(inds2):
#         for x in range(i,j):
#             used_data.insert(len(used_data)-1,(data_out2[x]))  

#     for x in used_data:
#         used_x.insert(len(used_x)-1,x[0])
#         used_y.insert(len(used_y)-1,x[1])  

#     # 3 mix
#     X_train_mix = np.setdiff1d(X_train, np.array(used_x))#53834, 64278
#     y_train_mix = np.setdiff1d(y_train, np.array(used_y))

#     print(X_train_mix.shape, X_train.shape)
#     print(type(X_train_mix), type(X_train))
#     print(type(data_out))
#     print(y_train_mix.shape, y_train.shape)

#     clients3, inds3 , data_out= create_hetero_clients(X_train, y_train, start_client = 2, num_clients=5, initial='client')
#     clients = {**clients, **clients3}
    
    return clients


In [16]:
def initializer_income_race_gender():
    
    female_features =  feature_set1[feature_set1['gender_Female']==1]
    female_features_above50 = female_features[female_features['income']==1]
    female_features_above50_white = female_features_above50[female_features_above50['race_White']==1]
    female_features_above50_black = female_features_above50[female_features_above50['race_Black']==1]
    
#     white, female, >50k
    x_female_white_more = female_features_above50_white[cols].copy().values
    y_female_white_more = female_features_above50_white[['income']].copy().values
    X_train_female_white_more = sc.fit_transform(x_female_white_more)
    y_train_female_white_more = y_female_white_more
    
#     black, female, >50k
    x_female_black_more = female_features_above50_black[cols].copy().values
    y_female_black_more = female_features_above50_black[['income']].copy().values
    X_train_female_black_more = sc.fit_transform(x_female_black_more)
    y_train_female_black_more = y_female_black_more
    
    female_features_below = female_features[female_features['income']==0]
    female_features_below_white = female_features_below[female_features_below['race_White']==1]
    female_features_below_black = female_features_below[female_features_below['race_Black']==1]
    
#    white, female, <=50k
    x_female_white_less = female_features_below_white[cols].copy().values
    y_female_white_less = female_features_below_white[['income']].copy().values
    X_train_female_white_less = sc.fit_transform(x_female_white_less)
    y_train_female_white_less = y_female_white_less
    
#    black, female, <=50k
    x_female_black_less = female_features_below_black[cols].copy().values
    y_female_black_less = female_features_below_black[['income']].copy().values
    X_train_female_black_less = sc.fit_transform(x_female_black_less)
    y_train_female_black_less = y_female_black_less
    


    male_features = feature_set1[feature_set1['gender_Male']==1] 
    male_features_above50 = male_features[male_features['income']==1]
    male_features_above50_white = male_features_above50[male_features_above50['race_White']==1]
    male_features_above50_black = male_features_above50[male_features_above50['race_Black']==1]
    
#     white, male, >50k
    x_male_white_more = male_features_above50_white[cols].copy().values
    y_male_white_more = male_features_above50_white[['income']].copy().values
    X_train_male_white_more = sc.fit_transform(x_male_white_more)
    y_train_male_white_more = y_male_white_more
    
#     black, male, >50k
    x_male_black_more = male_features_above50_black[cols].copy().values
    y_male_black_more = male_features_above50_black[['income']].copy().values
    X_train_male_black_more = sc.fit_transform(x_male_black_more)
    y_train_male_black_more = y_male_black_more
    
    male_features_below = male_features[male_features['income']==0]
    male_features_below_white = male_features_below[male_features_below['race_White']==1]
    male_features_below_black = male_features_below[male_features_below['race_Black']==1]
    
#    white, male, <=50k
    x_male_white_less = male_features_below_white[cols].copy().values
    y_male_white_less = male_features_below_white[['income']].copy().values
    X_train_male_white_less = sc.fit_transform(x_male_white_less)
    y_train_male_white_less = y_male_white_less
    
#    black, male, <=50k
    x_male_black_less = male_features_below_black[cols].copy().values
    y_male_black_less = male_features_below_black[['income']].copy().values
    X_train_male_black_less = sc.fit_transform(x_male_black_less)
    y_train_male_black_less = y_male_black_less
    
#     print("fwm: ", X_train_female_white_more.shape, y_train_female_white_more.shape,
#           "fbm: ", X_train_female_black_more.shape, y_train_female_black_more.shape, 
#             "fwl: ", X_train_female_white_less.shape, y_train_female_white_less.shape, 
#             "fbl: ", X_train_female_black_less.shape,  y_train_female_black_less.shape,
#             "mwm: ", X_train_male_white_more.shape, y_train_male_white_more.shape,
#             "mbm: ", X_train_male_black_more.shape, y_train_male_black_more.shape,
#             "mwl: ", X_train_male_white_less.shape, y_train_male_white_less.shape,
#             "mbl: ", X_train_male_black_less.shape, y_train_male_black_less.shape)

    
    return [X_train_female_white_more, y_train_female_white_more, 
            X_train_female_black_more, y_train_female_black_more, 
            X_train_female_white_less, y_train_female_white_less, 
            X_train_female_black_less, y_train_female_black_less,
            X_train_male_white_more, y_train_male_white_more,
            X_train_male_black_more, y_train_male_black_more,
            X_train_male_white_less, y_train_male_white_less,
            X_train_male_black_less, y_train_male_black_less]

In [17]:
# initializer_income_race_gender()

In [26]:
def get_hetero_clients_gender_race(train_sets):
    data = list(zip(train_sets[0], train_sets[1]))
    random.shuffle(data)
    
#     print(type(train_sets), type(train_sets[0]),type(list(list(zip(*data))[0]) ), type(data))
    
    train_sets[0] = list(list(zip(*data))[0])
    train_sets[1] = list(list(zip(*data))[1])
    data.clear()
    
    data = list(zip(train_sets[2], train_sets[3]))
    random.shuffle(data)
    
    train_sets[2], train_sets[3] = list(zip(*data))[0], list(zip(*data))[1]
    data.clear()
    data = list(zip(train_sets[4], train_sets[5]))
    random.shuffle(data)
    
    train_sets[4], train_sets[5] = list(zip(*data))[0], list(zip(*data))[1]
    data.clear()
    
    data = list(zip(train_sets[6], train_sets[7]))
    random.shuffle(data)
    
    train_sets[6], train_sets[7] = list(zip(*data))[0], list(zip(*data))[1]
    data.clear()
    
    data = list(zip(train_sets[8], train_sets[9]))
    random.shuffle(data)
    
    train_sets[8], train_sets[9] = list(zip(*data))[0], list(zip(*data))[1]
    data.clear()
    
    data = list(zip(train_sets[10], train_sets[11]))
    random.shuffle(data)
    
    train_sets[10], train_sets[11] = list(zip(*data))[0], list(zip(*data))[1]
    
    data = list(zip(train_sets[12], train_sets[13]))
    random.shuffle(data)
    
    train_sets[12], train_sets[13] = list(zip(*data))[0], list(zip(*data))[1]
    data.clear()
    
    data = list(zip(train_sets[14], train_sets[15]))
    random.shuffle(data)
    
    train_sets[14], train_sets[15] = list(zip(*data))[0], list(zip(*data))[1]
    
#     for i in range(7):

#         data = list(zip(train_sets[i], train_sets[i+1]))
#         random.shuffle(data)

# #         print(type(train_sets), type(train_sets[0]),type(list(list(zip(*data))[0]) ), type(data))

#         train_sets[i], train_sets[i+1] = list(zip(*data))[0], list(zip(*data))[1]
#         data.clear()
    
    
    clients, inds, data_out1 = create_hetero_clients(train_sets[0], train_sets[1], start_client = 0, num_clients=1, initial='client') 
    
    clients2, inds2, data_out2 = create_hetero_clients(train_sets[2], train_sets[3], start_client = 1, num_clients=2, initial='client')
    clients = {**clients, **clients2}
    
    clients3, inds3, data_out3 = create_hetero_clients(train_sets[4], train_sets[5], start_client = 2, num_clients=3, initial='client')
    clients = {**clients, **clients3}
    
    clients4, inds4, data_out4 = create_hetero_clients(train_sets[6], train_sets[7], start_client = 3, num_clients=4, initial='client')
    clients = {**clients, **clients4}
    
    clients5, inds5, data_out5 = create_hetero_clients(train_sets[8], train_sets[9], start_client = 4, num_clients=5, initial='client')
    clients = {**clients, **clients5}
    
    clients6, inds6, data_out6 = create_hetero_clients(train_sets[10], train_sets[11], start_client = 5, num_clients=6, initial='client')
    clients = {**clients, **clients6}
    
    clients7, inds7, data_out7 = create_hetero_clients(train_sets[12], train_sets[13], start_client = 6, num_clients=7, initial='client')
    clients = {**clients, **clients7}
    
    clients8, inds8, data_out8 = create_hetero_clients(train_sets[14], train_sets[15], start_client = 7, num_clients= 8, initial='client')
    clients = {**clients, **clients8}
    
    return clients


In [19]:
# take bs = 128 for 5 clients and 10 rounds
def batch_data(data_shard, bs=64):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    data = []
    label = []
    for x in data_shard:
        data.append(x[0])
        label.append(x[1])
    #seperate shard into data and labels lists
#     data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
#     print( label[0])
    return dataset.shuffle(len(label)).batch(bs)

In [20]:

#process and batch the training data for each client
def batch_clients(clients):
    clients_batched = dict()
    for (client_name, data) in clients.items():
#         print("data ",len(data))
        clients_batched[client_name] = batch_data(data)#non-IID
#         clients_batched[client_name] = batch_data(data,1) #IID
    

    #process and batch the test set  
    test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))
    
#     test_batched
    return clients_batched, test_batched


In [21]:
# clients = get_hetero_clients_gender(list(train_sets) )
# # print("client0: ", clients[0])

# clients_batched, test_batched = batch_clients(clients)
# print("client batched 0: ", len(clients_batched[0]))
# print("test" ,X_test.shape)
# train_list = initializer_income()
# for i in train_list:
#     print(i.shape)
    
# client_names= list(clients_batched.keys())

# for client in client_names:
#     print(client," ", len(clients_batched[client]))

In [22]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes=2 , learning_rate = 0.001, metric = "accuracy"):

        model = Sequential()
        
        model.add(Dense(128, input_shape = (shape,)))
#         model.add(Dense(128, Activation("relu")))
#         model.add(Dense(64, Activation("relu")))
#         model.add(Dense(32, Activation("relu")))
#         model.add(Dense(1))
        
        model.add(Dense(128, Activation("tanh")))
        model.add(Dense(64, Activation("tanh")))
        model.add(Dense(32, Activation("tanh")))
        model.add(Dense(1,Activation('sigmoid')))
        


        return model

In [23]:
lr = 0.005
comms_round = 10
loss=tf.keras.losses.BinaryCrossentropy(from_logits = False)

metrics = ['binary_accuracy']

optimizer = SGD(learning_rate=lr, 
                decay=lr / comms_round, 
                momentum=0.5
               )     



In [24]:


def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
#     print(len(scaled_weight_list))
    for grad_list_tuple in zip(*scaled_weight_list):
#         print(len(grad_list_tuple))
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    
#     cce = tf.keras.losses.BinaryCrossentropy(from_logits = True)
#     cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    logits = model.predict(X_test)

    score = global_model.evaluate(X_test, y_test, verbose=0)
    acc = score[1] ; loss = score[0]
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))

    return acc, loss

In [27]:

#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(X_train.shape[1] ,classes=2)
global_model.compile(optimizer=optimizer, loss=loss, metrics=metrics) 
client_names = [0, 1, 2, 3, 4, 5, 6, 7]
# client_names = [0, 1, 2, 3]
global_loss_list = []
global_freq_list = []
global_acc_list = []
client_loss = {i: np.array([]) for i in range(len(client_names))}
client_accuracy = {i: np.array([]) for i in range(len(client_names))}
client_frequency = {i: np.array([]) for i in range(len(client_names))}

# train_sets = initializer_income()
train_sets = initializer_income_race_gender()
# print("TRAIN SETS", len(train_sets))

#commence global training loop
for comm_round in range(comms_round):
    epoch_freq = 0
#     clients = create_clients(X_train, y_train, num_clients=15, initial='client')
#     clients = get_hetero_clients()
#     clients = get_hetero_clients_gender(list(train_sets) )
    clients = get_hetero_clients_gender_race(list(train_sets) )

    clients_batched, test_batched = batch_clients(clients)
#     print(len(clients_batched))
    
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(X_train.shape[1],classes=2)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        history = local_model.fit(clients_batched[client], epochs=1, verbose=1)
        
#         get client acc, loss
#         print(client)
        if(len(client_loss[client])== 0):
            client_loss[client] = [history.history['loss'][0]]
            client_accuracy[client] = [history.history['binary_accuracy'][0]]
            client_frequency[client] = [len(clients_batched[client])]
        
        else:
            client_loss[client] = np.append(client_loss[client], (history.history['loss'][0]))
            client_frequency[client] = np.append(client_frequency[client], len(clients_batched[client]))
            client_accuracy[client] = np.append(client_accuracy[client], (history.history['binary_accuracy'][0]))
        
        epoch_freq += len(clients_batched[client])
        
        print("client", client, "loss -->" ,client_loss[client], "freq->", client_frequency[client], "accuracy->", client_accuracy[client])
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)
    
    global_freq_list.append(epoch_freq)
    epoch_freq = 0
    

#     test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
        global_loss_list.append(global_loss)
        global_acc_list.append(global_acc)
        
        print("global_loss_list: ", global_loss_list )
        print("global_acc_list: ", global_acc_list )
        print("global_freq_list: ", global_freq_list)



<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


154/154 [==============================] - 0s 2ms/step - loss: 0.6520 - binary_accuracy: 0.6244
client 0 loss --> [0.6520453095436096] freq-> [154] accuracy-> [0.6243902444839478]
42/42 [==============================] - 0s 2ms/step - loss: 0.6949 - binary_accuracy: 0.5042
client 5 loss --> [0.6948809623718262] freq-> [42] accuracy-> [0.5041666626930237]
12/12 [==============================] - 0s 2ms/step - loss: 0.7229 - binary_accuracy: 0.4250
client 1 loss --> [0.722898006439209] freq-> [12] accuracy-> [0.42500001192092896]
900/900 [==============================] - 3s 3ms/step - loss: 0.4156 - binary_accuracy: 0.9039
client 4 loss --> [0.4156477153301239] freq-> [900] accuracy-> [0.9039409756660461]
214/214 [==============================] - 0s 2ms/step - loss: 0.6314 - binary_accuracy: 0.6750
client 3 loss --> [0.6314299702644348] freq-> [214] accuracy-> [0.675000011920929]
1921/1921 [==============================] - 6s 3ms/step - loss: 0.2977 - binary_accuracy: 0.9430
client 6 

321/321 [==============================] - 1s 3ms/step - loss: 0.1057 - binary_accuracy: 0.9972
client 3 loss --> [0.63142997 0.25575614 0.15227449 0.1191789  0.10566977] freq-> [214 241 268 294 321] accuracy-> [0.67500001 0.98908383 0.99619883 0.9968102  0.99717349]
1602/1602 [==============================] - 5s 3ms/step - loss: 0.0774 - binary_accuracy: 0.9988
client 7 loss --> [0.36101633 0.16614816 0.10835388 0.08735743 0.07736473] freq-> [1504 1529 1553 1577 1602] accuracy-> [0.92297381 0.99583846 0.99716187 0.99824613 0.99877048]
1350/1350 [==============================] - 4s 3ms/step - loss: 0.9111 - binary_accuracy: 0.4711
client 4 loss --> [0.41564772 0.71785086 0.85538751 0.89620405 0.91105551] freq-> [ 900 1013 1125 1238 1350] accuracy-> [0.90394098 0.54584879 0.458875   0.46171716 0.4710764 ]
2908/2908 [==============================] - 6s 2ms/step - loss: 0.0651 - binary_accuracy: 0.9990
client 6 loss --> [0.29766285 0.13327065 0.08817774 0.07168613 0.06509247] freq-> [1

401/401 [==============================] - 1s 3ms/step - loss: 0.0949 - binary_accuracy: 0.9966
client 3 loss --> [0.63142997 0.25575614 0.15227449 0.1191789  0.10566977 0.09996787
 0.09680084 0.09486126] freq-> [214 241 268 294 321 348 375 401] accuracy-> [0.67500001 0.98908383 0.99619883 0.9968102  0.99717349 0.99671614
 0.99678361 0.99656922]
289/289 [==============================] - 1s 3ms/step - loss: 1.9033 - binary_accuracy: 0.0053
client 0 loss --> [0.65204531 1.32216609 1.65442705 1.87752938 1.96135139 1.97029376
 1.94301367 1.90330005] freq-> [154 173 193 212 231 250 270 289] accuracy-> [0.62439024 0.01698284 0.00691057 0.00620843 0.00569106 0.00456535
 0.00481998 0.00525745]
78/78 [==============================] - 0s 3ms/step - loss: 2.2754 - binary_accuracy: 0.0024
client 5 loss --> [0.69488096 1.41669953 1.89260554 2.11936712 2.2073369  2.27492952
 2.28054476 2.27543497] freq-> [42 47 52 57 62 68 73 78] accuracy-> [0.50416666 0.00841751 0.00272727 0.         0.         0

2439/2439 [==============================] - 8s 3ms/step - loss: 0.0639 - binary_accuracy: 0.9994
client 2 loss --> [0.41144952 0.17041089 0.1078152  0.08696163 0.07697608 0.07115313
 0.06942742 0.06638124 0.06655346 0.06392072] freq-> [1148 1291 1435 1578 1722 1865 2009 2152 2295 2439] accuracy-> [0.90625    0.99518275 0.99690634 0.99779165 0.99872911 0.99901122
 0.99903518 0.99915034 0.99927831 0.99937844]
4140/4140 [==============================] - 16s 4ms/step - loss: 0.0557 - binary_accuracy: 0.9992
client 6 loss --> [0.29766285 0.13327065 0.08817774 0.07168613 0.06509247 0.06022613
 0.05800061 0.05786391 0.05624533 0.05568424] freq-> [1921 2168 2415 2661 2908 3154 3401 3647 3894 4140] accuracy-> [0.94304538 0.99666232 0.99784464 0.99869627 0.99902183 0.99897444
 0.99900287 0.99902314 0.999089   0.99917346]
455/455 [==============================] - 1s 3ms/step - loss: 0.0945 - binary_accuracy: 0.9959
client 3 loss --> [0.63142997 0.25575614 0.15227449 0.1191789  0.10566977 0.099

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats 

pdfs = []
for i in list(client_accuracy.keys()):
    print("client_",i, " loss: ", client_loss[i],"freq: ", client_frequency[i], "acc", client_accuracy[i])
    
#     norm
#     mean, std = stats.norm.fit(client_loss[i])
#     pdf =  stats.norm.pdf(client_loss[i], mean, std)

#     df_mean = np.mean(client_frequency[i])
#     df_std = np.std(client_frequency[i])
#     pdf = stats.norm.pdf(client_frequency[i], df_mean, df_std)
    
#     lognorm
#     shape, loc, scale = stats.lognorm.fit(client_loss[i])
#     pdf = stats.lognorm.pdf(client_loss[i], shape, loc, scale)

#     beta
#     beta_params = stats.beta.fit(client_loss[i])
#     pdf = stats.beta.pdf(client_loss[i], beta_params[0], beta_params[1], beta_params[2], beta_params[3])

#     burr
#     burr_params = stats.burr.fit(client_loss[i])
#     pdf = stats.burr.pdf(client_loss[i], burr_params[0], burr_params[1], burr_params[2], burr_params[3])

    
#  gamma
#     shape, loc, scale = stats.gamma.fit(client_loss[i])
#     pdf = stats.gamma.pdf(client_loss[i], shape, loc=loc, scale=scale)
    
    pdfs.append(pdf)
    sns.set_style("whitegrid")

    ax = sns.displot( x = pdf, kind = "kde", height=10, aspect=2,
                linewidth = 5 )
    plt.ylabel("Frequency")
#     plt.xlabel("Loss")
    plt.xlabel("Accuracy")
    
    plt.show()

In [29]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats 

for i in list(client_accuracy.keys()):
#     print("client_",i, "\nloss: ", client_loss[i],"\nfreq: ", client_frequency[i], "\nacc", client_accuracy[i])
    print("client_",i, " freq: ", client_frequency[i])    

client_ 0  freq:  [154 173 193 212 231 250 270 289 308 327]
client_ 1  freq:  [12 13 15 16 17 19 20 22 23 24]
client_ 2  freq:  [1148 1291 1435 1578 1722 1865 2009 2152 2295 2439]
client_ 3  freq:  [214 241 268 294 321 348 375 401 428 455]
client_ 4  freq:  [ 900 1013 1125 1238 1350 1463 1575 1688 1800 1913]
client_ 5  freq:  [42 47 52 57 62 68 73 78 83 88]
client_ 6  freq:  [1921 2168 2415 2661 2908 3154 3401 3647 3894 4140]
client_ 7  freq:  [1504 1529 1553 1577 1602 1626 1650 1675 1699 1724]


In [30]:
print("_GAMMA_")

pdfs_loss_gamma = []
pdfs_acc_gamma = []
pdfs_freq_gamma = []
for i in list(client_accuracy.keys()):
#     print("client_",i, "\nloss: ", client_loss[i],"\nfreq: ", client_frequency[i], "\nacc", client_accuracy[i])

    shape, loc, scale = stats.gamma.fit(client_loss[i])
    pdf_loss_gamma =  stats.gamma.pdf(client_loss[i], shape, loc, scale)
    pdfs_loss_gamma.append(pdf_loss_gamma)

    shape, loc, scale= stats.gamma.fit(client_accuracy[i])
    pdf_acc_gamma = stats.gamma.pdf(client_accuracy[i],shape, loc, scale)
    pdfs_acc_gamma.append(pdf_acc_gamma)

    shape, loc, scale  = stats.gamma.fit(client_frequency[i])
    pdf_freq_gamma = stats.gamma.pdf(client_frequency[i],shape, loc, scale )
    pdfs_freq_gamma.append(pdf_freq_gamma)


print("ACCURACY")
for i in range(len(pdfs_acc_gamma)):
    for j in range(i+1,len(pdfs_acc_gamma)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc_gamma[i], pdfs_acc_gamma[j]))


shape, loc, scale = stats.gamma.fit(global_acc_list)
pdf_global_acc_gamma =  stats.gamma.pdf(global_acc_list, shape, loc, scale)

for i in range(len(pdfs_acc_gamma)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc_gamma[i], pdf_global_acc_gamma))

print("LOSS")
for i in range(len(pdfs_loss_gamma)):
    for j in range(i+1,len(pdfs_loss_gamma)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss_gamma[i], pdfs_loss_gamma[j]))


shape, loc, scale = stats.gamma.fit(global_loss_list)
pdf_global_loss_gamma =  stats.gamma.pdf(global_loss_list, shape, loc, scale)

for i in range(len(pdfs_loss_gamma)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss_gamma[i], pdf_global_loss_gamma))

print("FREQUENCY")
for i in range(len(pdfs_freq_gamma)):
    for j in range(i+1,len(pdfs_freq_gamma)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq_gamma[i], pdfs_freq_gamma[j]))


shape, loc, scale = stats.gamma.fit(global_freq_list)
pdf_global_freq_gamma =  stats.gamma.pdf(global_freq_list,shape, loc, scale)

for i in range(len(pdfs_freq_gamma)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq_gamma[i], pdf_global_freq_gamma))

_GAMMA_
ACCURACY
0  and  1 3.5001817549856585e+23
0  and  2 7869143271.861214
0  and  3 357034.07862304547
0  and  4 518126.9362475103
0  and  5 165104.89760287738
0  and  6 19103681246.857594
0  and  7 392016643.4018573
1  and  2 3.50018175498558e+23
1  and  3 3.5001817549856585e+23
1  and  4 3.5001817549856585e+23
1  and  5 3.5001817549856585e+23
1  and  6 3.500181754985467e+23
1  and  7 3.500181754985655e+23
2  and  3 7868786239.209446
2  and  4 7868625147.615849
2  and  5 7869231109.581604
2  and  6 11234537976.059536
2  and  7 7477126628.530401
3  and  4 161100.1784975285
3  and  5 444871.7990126755
3  and  6 19103324215.268982
3  and  7 391659611.30970186
4  and  5 605962.197813923
4  and  6 19103163122.99236
4  and  7 391498519.3510584
5  and  6 19103769084.577984
5  and  7 392104481.1222469
6  and  7 18711664603.959282
0  and global:  41259533824.28781
1  and global:  3.5001817549852465e+23
2  and global:  33390390564.280167
3  and global:  41259176803.48961
4  and global:  412

In [ ]:

for i in range(len(pdfs_loss_gamma)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss_gamma[i], pdf_global_loss_gamma))
    M = ot.dist(pdfs_loss_gamma[i], pdf_global_loss_gamma, metric='euclidean')
    W = ot.emd2(pdfs_loss_gamma[i], pdf_global_loss_gamma, M)
    print("actual ", W)

In [31]:
print("_NORM_")

pdfs_loss_norm = []
pdfs_acc_norm = []
pdfs_freq_norm = []
for i in list(client_accuracy.keys()):
#     print("client_",i, " loss: ", client_loss[i],"freq: ", client_frequency[i], "acc", client_accuracy[i])
    
    mean, std = stats.norm.fit(client_loss[i])
    pdf_loss_norm =  stats.norm.pdf(client_loss[i], mean, std)
    pdfs_loss_norm.append(pdf_loss_norm)

    mean, std = stats.norm.fit(client_accuracy[i])
    pdf_acc_norm = stats.norm.pdf(client_accuracy[i],mean, std )
    pdfs_acc_norm.append(pdf_acc_norm)
    
    mean, std  = stats.norm.fit(client_frequency[i])
    pdf_freq_norm = stats.norm.pdf(client_frequency[i], mean, std )
    pdfs_freq_norm.append(pdf_freq_norm)
    
    
print("ACCURACY")
for i in range(len(pdfs_acc_norm)):
    for j in range(i+1,len(pdfs_acc_norm)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc_norm[i], pdfs_acc_norm[j]))
        
        
mean, std = stats.norm.fit(global_acc_list)
pdf_global_acc_norm =  stats.norm.pdf(global_acc_list, mean, std)

for i in range(len(pdfs_acc_norm)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc_norm[i], pdf_global_acc_norm))
    
print("LOSS")
for i in range(len(pdfs_loss_norm)):
    for j in range(i+1,len(pdfs_loss_norm)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss_norm[i], pdfs_loss_norm[j]))
        
        
mean, std = stats.norm.fit(global_loss_list)
pdf_global_loss_norm =  stats.norm.pdf(global_loss_list, mean, std)

for i in range(len(pdfs_loss_norm)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss_norm[i], pdf_global_loss_norm))
    
print("FREQUENCY")
for i in range(len(pdfs_freq_norm)):
    for j in range(i+1,len(pdfs_freq_norm)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq_norm[i], pdfs_freq_norm[j]))
        
        
mean, std = stats.norm.fit(global_freq_list)
pdf_global_freq_norm =  stats.norm.pdf(global_freq_list, mean, std)

for i in range(len(pdfs_freq_norm)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq_norm[i], pdf_global_freq_norm))


_NORM_
ACCURACY
0  and  1 0.8325300686638157
0  and  2 10.459736423104237
0  and  3 1.698031063319809
0  and  4 0.6592378845059876
0  and  5 0.42290213571143564
0  and  6 18.528715177097837
0  and  7 13.219825443615902
1  and  2 9.627206354440421
1  and  3 0.8655009946559932
1  and  4 0.32002245930806605
1  and  5 0.4096279329523801
1  and  6 17.696185108434015
1  and  7 12.387295374952084
2  and  3 8.761705359784427
2  and  4 9.800498538598248
2  and  5 10.036834287392804
2  and  6 8.068978753993596
2  and  7 2.7600890205116646
3  and  4 1.0479685475623837
3  and  5 1.275128927608373
3  and  6 16.830684113778027
3  and  7 11.521794380296091
4  and  5 0.41552401074085465
4  and  6 17.86947729259185
4  and  7 12.560587559109912
5  and  6 18.105813041386398
5  and  7 12.796923307904466
6  and  7 5.308889733481931
0  and global:  116.55554794469576
1  and global:  115.72301787603195
2  and global:  106.09581152159153
3  and global:  114.85751688137596
4  and global:  115.89631006018978
5 

In [32]:
print("_LOGNORM_")

pdfs_loss = []
pdfs_acc = []
pdfs_freq = []
for i in list(client_accuracy.keys()):
#     print("client_",i, "\nloss: ", client_loss[i],"\nfreq: ", client_frequency[i], "\nacc", client_accuracy[i])

    shape, loc, scale = stats.lognorm.fit(client_loss[i])
    pdf_loss =  stats.lognorm.pdf(client_loss[i], shape, loc, scale)
    pdfs_loss.append(pdf_loss)

    shape, loc, scale= stats.lognorm.fit(client_accuracy[i])
    pdf_acc = stats.lognorm.pdf(client_accuracy[i],shape, loc, scale)
    pdfs_acc.append(pdf_acc)

    shape, loc, scale  = stats.lognorm.fit(client_frequency[i])
    pdf_freq = stats.lognorm.pdf(client_frequency[i],shape, loc, scale )
    pdfs_freq.append(pdf_freq)


print("ACCURACY")
for i in range(len(pdfs_acc)):
    for j in range(i+1,len(pdfs_acc)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc[i], pdfs_acc[j]))


shape, loc, scale = stats.lognorm.fit(global_acc_list)
pdf_global_acc =  stats.lognorm.pdf(global_acc_list, shape, loc, scale)

for i in range(len(pdfs_acc)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc[i], pdf_global_acc))

print("LOSS")
for i in range(len(pdfs_loss)):
    for j in range(i+1,len(pdfs_loss)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss[i], pdfs_loss[j]))


shape, loc, scale = stats.lognorm.fit(global_loss_list)
pdf_global_loss =  stats.lognorm.pdf(global_loss_list, shape, loc, scale)

for i in range(len(pdfs_loss)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss[i], pdf_global_loss))

print("FREQUENCY")
for i in range(len(pdfs_freq)):
    for j in range(i+1,len(pdfs_freq)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq[i], pdfs_freq[j]))


shape, loc, scale = stats.lognorm.fit(global_freq_list)
pdf_global_freq =  stats.lognorm.pdf(global_freq_list,shape, loc, scale)

for i in range(len(pdfs_freq)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq[i], pdf_global_freq))

_LOGNORM_
ACCURACY
0  and  1 6.109867828617498e+18
0  and  2 79709480807.47874
0  and  3 79709480814.391
0  and  4 79709480810.03777
0  and  5 1.6971964477274035e+17
0  and  6 79709480801.30351
0  and  7 79709480805.46625
1  and  2 6.10986790832698e+18
1  and  3 6.10986790832698e+18
1  and  4 6.10986790832698e+18
1  and  5 5.940148183844758e+18
1  and  6 6.10986790832698e+18
1  and  7 6.10986790832698e+18
2  and  3 8.798886019579744
2  and  4 9.620161192220916
2  and  5 1.6971972448222112e+17
2  and  6 8.064896234963543
2  and  7 2.608621289287474
3  and  4 6.763128366138591
3  and  5 1.6971972448222112e+17
3  and  6 16.863782254543285
3  and  7 11.407507308867217
4  and  5 1.6971972448222112e+17
4  and  6 14.00588899954047
4  and  7 11.04412293698373
5  and  6 1.6971972448222112e+17
5  and  7 1.6971972448222112e+17
6  and  7 5.456274945676069
0  and global:  79709480793.47607
1  and global:  6.10986790832698e+18
2  and global:  25.754946932579383
3  and global:  34.55383295215913
4  a

In [33]:
print("_BETA_")

pdfs_loss = []
pdfs_acc = []
pdfs_freq = []
for i in list(client_accuracy.keys()):

    beta_params = stats.beta.fit(client_loss[i])
    pdf_loss =  stats.beta.pdf(client_loss[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_loss.append(pdf_loss)

    beta_params = stats.beta.fit(client_accuracy[i])
    pdf_acc = stats.beta.pdf(client_accuracy[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_acc.append(pdf_acc)

    beta_params  = stats.beta.fit(client_frequency[i])
    pdf_freq = stats.beta.pdf(client_frequency[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_freq.append(pdf_freq)


print("ACCURACY")
for i in range(len(pdfs_acc)):
    for j in range(i+1,len(pdfs_acc)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc[i], pdfs_acc[j]))


beta_params = stats.beta.fit(global_acc_list)
pdf_global_acc =  stats.beta.pdf(global_acc_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_acc)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc[i], pdf_global_acc))

print("LOSS")
for i in range(len(pdfs_loss)):
    for j in range(i+1,len(pdfs_loss)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss[i], pdfs_loss[j]))


beta_params = stats.beta.fit(global_loss_list)
pdf_global_loss =  stats.beta.pdf(global_loss_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_loss)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss[i], pdf_global_loss))

print("FREQUENCY")
for i in range(len(pdfs_freq)):
    for j in range(i+1,len(pdfs_freq)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq[i], pdfs_freq[j]))


beta_params = stats.beta.fit(global_freq_list)
pdf_global_freq =  stats.beta.pdf(global_freq_list,beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_freq)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq[i], pdf_global_freq))

_BETA_


C:\Users\riash\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
C:\Users\riash\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The number of calls to function has reached maxfev = 600.
  warnings.warn(msg, RuntimeWarning)
C:\Users\riash\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


ACCURACY
0  and  1 7.307236233053098e+28
0  and  2 37600461648677.42
0  and  3 48198812766773.0
0  and  4 48968916937776.31
0  and  5 48844505240609.46
0  and  6 48978053403745.07
0  and  7 44051455739019.99
1  and  2 7.307236233053103e+28
1  and  3 7.3072362330531045e+28
1  and  4 7.307236233053104e+28
1  and  5 7.307236233053104e+28
1  and  6 7.307236233053104e+28
1  and  7 7.307236233053102e+28
2  and  3 10598351118113.635
2  and  4 11368455289108.766
2  and  5 11244043591950.123
2  and  6 11377591755067.637
2  and  7 6450994090353.064
3  and  4 770104171003.5665
3  and  5 645692473836.4873
3  and  6 779240636972.3181
3  and  7 4147357027763.732
4  and  5 124411697167.80614
4  and  6 9136465968.751783
4  and  7 4917461198761.089
5  and  6 133548163136.5579
5  and  7 4793049501600.219
6  and  7 4926597664725.075
0  and global:  238239896020545.4
1  and global:  7.307236233053075e+28
2  and global:  275840357669204.75
3  and global:  286438708787318.3
4  and global:  287208812958313.2

In [34]:
print("_BURR_")

pdfs_loss = []
pdfs_acc = []
pdfs_freq = []
for i in list(client_accuracy.keys()):

    params = stats.burr.fit(client_loss[i])
    pdf_loss =  stats.burr.pdf(client_loss[i],params[0], params[1], params[2], params[3])
    pdfs_loss.append(pdf_loss)

    beta_params = stats.burr.fit(client_accuracy[i])
    pdf_acc = stats.burr.pdf(client_accuracy[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_acc.append(pdf_acc)

    beta_params  = stats.burr.fit(client_frequency[i])
    pdf_freq = stats.burr.pdf(client_frequency[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_freq.append(pdf_freq)


print("ACCURACY")
for i in range(len(pdfs_acc)):
    for j in range(i+1,len(pdfs_acc)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc[i], pdfs_acc[j]))


beta_params = stats.burr.fit(global_acc_list)
pdf_global_acc =  stats.burr.pdf(global_acc_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_acc)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc[i], pdf_global_acc))

print("LOSS")
for i in range(len(pdfs_loss)):
    for j in range(i+1,len(pdfs_loss)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss[i], pdfs_loss[j]))


beta_params = stats.burr.fit(global_loss_list)
pdf_global_loss =  stats.burr.pdf(global_loss_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_loss)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss[i], pdf_global_loss))

print("FREQUENCY")
for i in range(len(pdfs_freq)):
    for j in range(i+1,len(pdfs_freq)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq[i], pdfs_freq[j]))


beta_params = stats.burr.fit(global_freq_list)
pdf_global_freq =  stats.burr.pdf(global_freq_list,beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_freq)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq[i], pdf_global_freq))

_BURR_


C:\Users\riash\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:942: RuntimeWarning: overflow encountered in power
  - sc.xlog1py(d_+1, x_**(-c_))))
C:\Users\riash\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:929: RuntimeWarning: overflow encountered in power
  f2 = lambda x_, c_, d_: (c_ * d_ * (x_ ** (-c_ - 1.0)) /
C:\Users\riash\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:930: RuntimeWarning: overflow encountered in power
  ((1 + x_ ** (-c_)) ** (d_ + 1.0))))
C:\Users\riash\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:929: RuntimeWarning: invalid value encountered in true_divide
  f2 = lambda x_, c_, d_: (c_ * d_ * (x_ ** (-c_ - 1.0)) /


ACCURACY
0  and  1 9.632467572128582e+27
0  and  2 41444361615586.05
0  and  3 41444361615596.75
0  and  4 41444361615607.33
0  and  5 4.481935523269599e+19
0  and  6 41444361615584.23
0  and  7 41444361615585.336
1  and  2 9.632467572128622e+27
1  and  3 9.632467572128622e+27
1  and  4 9.632467572128622e+27
1  and  5 9.632467527309225e+27
1  and  6 9.632467572128622e+27
1  and  7 9.632467572128622e+27
2  and  3 17.405240826672046
2  and  4 30.113678715951945
2  and  5 4.481939667705761e+19
2  and  6 6.109357347514156
2  and  7 2.644867905478688
3  and  4 12.724712292977225
3  and  5 4.481939667705761e+19
3  and  6 23.5145981741862
3  and  7 20.050108732150733
4  and  5 4.481939667705761e+19
4  and  6 36.20399101750616
4  and  7 32.74869276729836
5  and  6 4.481939667705761e+19
5  and  7 4.481939667705761e+19
6  and  7 3.4644894420354695
0  and global:  41444361615583.23
1  and global:  9.632467572128622e+27
2  and global:  18.88400310048125
3  and global:  36.2892439271533
4  and glob

In [ ]:
for i in list(client_loss.keys()):
#     if(i== len(list(client_loss.keys()))-1):
#         continue
    print("client_",i, "loss: ", client_loss[i])

In [ ]:
for i in list(client_loss.keys()):
#     if(i== len(list(client_loss.keys()))-1):
#         continue
    print("client_",i, "freq: ", client_frequency[i])

In [ ]:
for i in range(len(pdfs)):
    for j in range(i+1,len(pdfs)):
        print(i," and ", j , stats.wasserstein_distance(pdfs[i], pdfs[j]))

In [ ]:
# # gamma
shape, loc, scale = stats.gamma.fit(global_freq_list)
pdf_global = stats.gamma.pdf(global_freq_list, shape, loc=loc, scale=scale)

# df_mean = np.mean(global_loss_list)
# df_std = np.std(global_loss_list)
# pdf_global = stats.norm.pdf(global_loss_list, df_mean, df_std)

# mean, std = stats.norm.fit(global_freq_list)
# pdf_global =  stats.norm.pdf(global_freq_list, mean, std)

# lognorm
# shape, loc, scale = stats.lognorm.fit(global_loss_list)
# pdf_global = stats.lognorm.pdf(global_loss_list, shape, loc, scale)

# global_acc_list
# global_freq_list

# burr
# burr_params = stats.beta.fit(global_freq_list)
# pdf_global = stats.beta.pdf(global_freq_list, burr_params[0], burr_params[1], burr_params[2], burr_params[3])

for i in range(len(pdfs)):
    print(i," and global: " , stats.wasserstein_distance(pdfs[i], pdf_global))

In [ ]:
!pip install fitter

In [ ]:
# from fitter import Fitter, get_common_distributions, get_distributions

# for i in list(client_loss.keys()):
#     height = client_loss[i]

#     f = Fitter(height,
#                distributions=['gamma',
#                               'lognorm',
#                               "beta",
#                               "burr",
#                               "norm"])
#     f.fit()
#     f.summary()

#     sns.set_style('white')
#     sns.set_context("paper", font_scale = 2)
#     sns.displot(data=dataset, x="Height", kind="hist", bins = 100, aspect = 1.5)

In [ ]:
# 8 clients---> gender, race, income
pdfs = []
for i in list(client_loss.keys()):
    if(i== len(list(client_loss.keys()))-1):
        continue
    print("client_",i, " loss: ", client_loss[i],"freq: ", client_frequency[i])

#     shape, loc, scale = stats.gamma.fit(client_loss[i])
    
#     # Printing the estimated parameters
#     print("Shape:", shape)
#     print("Location:", loc)
#     print("Scale:", scale)
#     pdf = stats.gamma.pdf(client_loss[i], shape, loc=loc, scale=scale)
    
    
    df_mean = np.mean(client_loss[i])
    df_std = np.std(client_loss[i])
    pdf = stats.norm.pdf(client_loss[i], df_mean, df_std)
    pdfs.append(pdf)
    sns.set_style("whitegrid")

    ax = sns.displot( x = pdf, kind = "kde", height=10, aspect=2,
                linewidth = 5 )
    ax.fig.suptitle('Original distribution', size = 20)
    plt.ylabel("Frequency")
    plt.xlabel("Loss")
    plt.show()

In [ ]:
for i in range(len(pdfs)):
    for j in range(i+1,len(pdfs)):
        print(i," and ", j , stats.wasserstein_distance(pdfs[i], pdfs[j]))

In [ ]:
shape, loc, scale = stats.gamma.fit(global_loss_list)
pdf_global = stats.gamma.pdf(global_loss_list, shape, loc=loc, scale=scale)

for i in range(len(pdfs)):
    print(i," and global: " , stats.wasserstein_distance(pdfs[i], pdf_global))

In [ ]:
# experiment plotter

# for i in list(client_loss.keys()):
#     if(i== len(list(client_loss.keys()))-1):
#         continue
#     print("client_",i, " loss: ", client_loss[i],"freq: ", client_frequency[i])
    
# #     df_mean = np.mean(client_loss[i])
# #     df_std = np.std(client_loss[i])
# #     pdf = stats.norm.pdf(client_loss[i], df_mean, df_std)

#     shape, loc, scale = stats.gamma.fit(client_loss[i])
    
#     # Printing the estimated parameters
# #     print("Shape:", shape)
# #     print("Location:", loc)
# #     print("Scale:", scale)
#     pdf = stats.gamma.pdf(client_loss[i], shape, loc=loc, scale=scale)

# #     plt.plot(client_loss[i], pdf, "-o", label = i)

# #     client_loss[i] = stats.gamma.rvs(1, size=5000)+5
#     sns.set_style("whitegrid")

#     ax = sns.displot( x = pdf, kind = "kde", height=10, aspect=2,
#                 linewidth = 5 )
#     ax.fig.suptitle('Original distribution', size = 20)
# #     plt.plot(client_loss[i], client_frequency[i], "-o", label = i)
# #     plt.legend()
#     plt.ylabel("Frequency")
#     plt.xlabel("Loss")
#     plt.show()

In [ ]:
score = global_model.evaluate(X_test, y_test, verbose=0)
score

In [ ]:
nn_preds = global_model.predict(X_test)
nn_preds = (nn_preds > 0.5)

nn_precision =precision_score(y_test, nn_preds)
nn_recall = recall_score(y_test, nn_preds)
nn_accuracy = accuracy_score(y_test, nn_preds)
nn_f1 = f1_score(y_test, nn_preds)


print("Precision = {}".format(nn_precision))
print("Recall = {}".format(nn_recall))
print("Accuracy = {}".format(nn_accuracy))
print("f1 = {}".format(nn_f1))


In [ ]:
arr = nn_preds > 0.5

unique, counts = np.unique(arr, return_counts=True)

np.asarray((unique, counts)).T

In [ ]:
# pdf for client losses
# x-loss
# y- frequency
# each client has one pdf for all rounds
# using histogram


# drop client
# non iid
# fedavg


# gender dist
# kernel density


# non-iid, loss to accuracy
# 4client